In [ ]:
import packages

In [ ]:
import os
import shutil
from typing import List, Optional, Union
from pathlib import Path


class PathCopier:
    """Handles recursive copying of folders and files while maintaining directory structure."""
    
    def __init__(self, source_paths: List[str], destination: str):
        """
        Initialize the PathCopier with source paths and destination.
        
        Args:
            source_paths: List of source paths (folders or files) to copy
            destination: Destination folder path
        """
        self.source_paths = [Path(path) for path in source_paths]
        self.destination = Path(destination)
        
    def validate_paths(self) -> Optional[str]:
        """
        Validate source and destination paths.
        
        Returns:
            Error message if validation fails, None otherwise
        """
        # Check if source paths exist
        for path in self.source_paths:
            if not path.exists():
                return f"Source path does not exist: {path}"
                
        # Remove destination if it exists and recreate it fresh
        if self.destination.exists():
            if not self.destination.is_dir():
                return f"Destination path exists but is not a directory: {self.destination}"
            try:
                shutil.rmtree(self.destination)
            except Exception as e:
                return f"Failed to remove existing destination folder: {str(e)}"
        
        try:
            self.destination.mkdir(parents=True)
        except Exception as e:
            return f"Failed to create destination folder: {str(e)}"
        
        return None

    def find_common_parent(self) -> Optional[Path]:
        """
        Find the common parent directory of all source paths.
        
        Returns:
            Path to common parent directory or None if no common parent
        """
        try:
            # Get the parts of each path
            path_parts_list = [list(path.parts) for path in self.source_paths]
            
            # Find the index where paths start to differ
            min_length = min(len(parts) for parts in path_parts_list)
            common_parts = []
            
            for i in range(min_length):
                current_part = path_parts_list[0][i]
                if all(parts[i] == current_part for parts in path_parts_list):
                    common_parts.append(current_part)
                else:
                    break
            
            if not common_parts:
                return None
                
            return Path(*common_parts)
            
        except Exception:
            return None

    def get_relative_path(self, source_path: Path, common_parent: Path) -> Path:
        """
        Get the relative path from common parent to source path.
        
        Args:
            source_path: Source path
            common_parent: Common parent path
            
        Returns:
            Relative path from common parent
        """
        try:
            return source_path.relative_to(common_parent.parent)
        except ValueError:
            # Fallback to just the name if relative_to fails
            return Path(source_path.name)

    def copy_path(self, source: Path, dest_path: Path) -> None:
        """
        Copy a file or folder to destination.
        
        Args:
            source: Source path
            dest_path: Destination path including relative structure
        """
        dest_full_path = self.destination / dest_path
        
        if source.is_file():
            # Create parent directories if needed
            dest_full_path.parent.mkdir(parents=True, exist_ok=True)
            # Copy the file
            shutil.copy2(source, dest_full_path)
        else:
            # It's a directory
            if dest_full_path.exists():
                shutil.rmtree(dest_full_path)
            # Copy the directory
            shutil.copytree(source, dest_full_path)
                
    def execute(self) -> Optional[str]:
        """
        Execute the copying operation.
        
        Returns:
            Error message if operation fails, None otherwise
        """
        # Validate paths first
        error = self.validate_paths()
        if error:
            return error
            
        try:
            # Find common parent
            common_parent = self.find_common_parent()
            if not common_parent:
                return "No common parent directory found in source paths"
                
            # Process each source path
            for source_path in self.source_paths:
                relative_path = self.get_relative_path(source_path, common_parent)
                self.copy_path(source_path, relative_path)
                
            # After copying, check and flatten if needed
            error = self.flatten_if_single_child()
            if error:
                return error
                
            return None
            
        except Exception as e:
            return f"Error during copy operation: {str(e)}"
            
    def flatten_if_single_child(self) -> Optional[str]:
        """
        If destination has only one child folder, move its contents up.
        
        Returns:
            Error message if operation fails, None otherwise
        """
        try:
            # Get immediate children
            children = list(self.destination.iterdir())
            
            # Check if there's exactly one child and it's a directory
            if len(children) == 1 and children[0].is_dir():
                single_child = children[0]
                
                # Create temporary directory for moving
                temp_dir = self.destination.parent / f"{self.destination.name}_temp"
                temp_dir.mkdir()
                
                try:
                    # Move all contents to temp directory
                    for item in single_child.iterdir():
                        shutil.move(str(item), str(temp_dir))
                    
                    # Remove the now-empty child directory
                    single_child.rmdir()
                    
                    # Move everything back from temp to destination
                    for item in temp_dir.iterdir():
                        shutil.move(str(item), str(self.destination))
                        
                    # Remove temp directory
                    temp_dir.rmdir()
                    
                except Exception as e:
                    # Clean up temp directory if it exists
                    if temp_dir.exists():
                        shutil.rmtree(temp_dir)
                    raise e
                    
            return None
            
        except Exception as e:
            return f"Error during structure flattening: {str(e)}"


def copy_paths(source_paths: List[str], destination: str) -> Optional[str]:
    """
    Copy multiple paths (files or folders) recursively to a destination while preserving structure.
    
    Args:
        source_paths: List of source paths (files or folders) to copy
        destination: Destination folder path
        
    Returns:
        Error message if operation fails, None otherwise
    
    Example:
        >>> source_paths = [
        ...     "/path/to/apps/assets",
        ...     "/path/to/apps/configs/config.json",
        ...     "/path/to/apps/context",
        ...     "/path/to/apps/data/org/data.txt"
        ... ]
        >>> error = copy_paths(source_paths, "/path/to/destination")
        >>> if error:
        >>>     print(f"Copy failed: {error}")
        >>> else:
        >>>     print("Copy completed successfully")
    """
    copier = PathCopier(source_paths, destination)
    return copier.execute()
  
# Example usage
source_folders = [
    f"{packages.ROOT_PATH}/apps/context/app/p3.yaml", #

]
destination = f"{packages.ROOT_PATH}/clone"

error = copy_paths(source_folders, destination)
if error:
    print(f"Copy failed: {error}")
else:
    print("Copy completed successfully")

In [ ]:
import os
import shutil
from typing import List, Optional, Union, Set
from pathlib import Path


class PathCopier:
    """Handles recursive copying of folders and files while maintaining directory structure."""
    
    def __init__(self, source_paths: List[str], destination: str, ignore: List[str] = None):
        """
        Initialize the PathCopier with source paths and destination.
        
        Args:
            source_paths: List of source paths (folders or files) to copy
            destination: Destination folder path
            ignore: List of file or folder names or full paths to ignore during copy
        """
        self.source_paths = [Path(path) for path in source_paths]
        self.destination = Path(destination)
        
        # Convert all ignore paths to Path objects for easier comparison
        self.ignore = set()
        if ignore:
            for item in ignore:
                self.ignore.add(Path(item))
        
    def validate_paths(self) -> Optional[str]:
        """
        Validate source and destination paths.
        
        Returns:
            Error message if validation fails, None otherwise
        """
        # Check if source paths exist
        for path in self.source_paths:
            if not path.exists():
                return f"Source path does not exist: {path}"
                
        # Remove destination if it exists and recreate it fresh
        if self.destination.exists():
            if not self.destination.is_dir():
                return f"Destination path exists but is not a directory: {self.destination}"
            try:
                shutil.rmtree(self.destination)
            except Exception as e:
                return f"Failed to remove existing destination folder: {str(e)}"
        
        try:
            self.destination.mkdir(parents=True)
        except Exception as e:
            return f"Failed to create destination folder: {str(e)}"
        
        return None

    def find_common_parent(self) -> Optional[Path]:
        """
        Find the common parent directory of all source paths.
        
        Returns:
            Path to common parent directory or None if no common parent
        """
        try:
            # Get the parts of each path
            path_parts_list = [list(path.parts) for path in self.source_paths]
            
            # Find the index where paths start to differ
            min_length = min(len(parts) for parts in path_parts_list)
            common_parts = []
            
            for i in range(min_length):
                current_part = path_parts_list[0][i]
                if all(parts[i] == current_part for parts in path_parts_list):
                    common_parts.append(current_part)
                else:
                    break
            
            if not common_parts:
                return None
                
            return Path(*common_parts)
            
        except Exception:
            return None

    def get_relative_path(self, source_path: Path, common_parent: Path) -> Path:
        """
        Get the relative path from common parent to source path.
        
        Args:
            source_path: Source path
            common_parent: Common parent path
            
        Returns:
            Relative path from common parent
        """
        try:
            return source_path.relative_to(common_parent.parent)
        except ValueError:
            # Fallback to just the name if relative_to fails
            return Path(source_path.name)

    def should_ignore(self, path: Path) -> bool:
        """
        Check if the path should be ignored based on the ignore list.
        
        Args:
            path: Path to check
            
        Returns:
            True if path should be ignored, False otherwise
        """
        # Check if the exact path is in the ignore list
        if path in self.ignore:
            return True
        
        # Check if the absolute path is in the ignore list
        if path.absolute() in self.ignore:
            return True
            
        # Check if the file or folder name is in the ignore list (simple name match)
        for ignore_path in self.ignore:
            # Match by name only (for simple file/folder names like "node_modules")
            if ignore_path.name == path.name:
                return True
                
        # Check all parent directories in case a folder was specified
        for parent in path.parents:
            if parent.name in {ignore_path.name for ignore_path in self.ignore}:
                return True
                
        # For each path segment, check if any parts of the path match
        path_parts = list(path.parts)
        for ignore_path in self.ignore:
            ignore_parts = list(ignore_path.parts)
            
            # Check if the entire ignore path is a subset of this path
            path_str = str(path.absolute())
            ignore_str = str(ignore_path)
            
            # If the path ends with the ignore path or contains it fully
            if path_str.endswith(ignore_str) or ignore_str in path_str:
                # Make sure it's a proper match (not just a substring of a filename)
                # This check handles cases where the ignore path is a full path
                if os.path.dirname(path_str).endswith(os.path.dirname(ignore_str)) or \
                   path.name == ignore_path.name:
                    return True
                
        return False

    def copy_path(self, source: Path, dest_path: Path) -> None:
        """
        Copy a file or folder to destination, skipping ignored files/folders.
        
        Args:
            source: Source path
            dest_path: Destination path including relative structure
        """
        # Skip if this path should be ignored
        if self.should_ignore(source):
            print(f"Ignoring: {source}")
            return
            
        dest_full_path = self.destination / dest_path
        
        if source.is_file():
            # Create parent directories if needed
            dest_full_path.parent.mkdir(parents=True, exist_ok=True)
            # Copy the file
            shutil.copy2(source, dest_full_path)
        else:
            # It's a directory - do a custom copy to handle ignored files
            self.copy_directory(source, dest_full_path)
                
    def copy_directory(self, source_dir: Path, dest_dir: Path) -> None:
        """
        Custom directory copying that respects the ignore list.
        
        Args:
            source_dir: Source directory path
            dest_dir: Destination directory path
        """
        # Create the destination directory
        dest_dir.mkdir(parents=True, exist_ok=True)
        
        # Copy each item in the directory, respecting ignore list
        for item in source_dir.iterdir():
            # Skip if this item should be ignored
            if self.should_ignore(item):
                print(f"Ignoring: {item}")
                continue
                
            dest_item = dest_dir / item.name
            
            if item.is_file():
                shutil.copy2(item, dest_item)
            else:
                # Recursively copy subdirectories
                self.copy_directory(item, dest_item)
                
    def execute(self) -> Optional[str]:
        """
        Execute the copying operation.
        
        Returns:
            Error message if operation fails, None otherwise
        """
        # Validate paths first
        error = self.validate_paths()
        if error:
            return error
            
        try:
            # Find common parent
            common_parent = self.find_common_parent()
            if not common_parent:
                return "No common parent directory found in source paths"
                
            # Process each source path
            for source_path in self.source_paths:
                # Skip if this path should be ignored
                if self.should_ignore(source_path):
                    print(f"Ignoring: {source_path}")
                    continue
                    
                relative_path = self.get_relative_path(source_path, common_parent)
                self.copy_path(source_path, relative_path)
                
            # After copying, check and flatten if needed
            error = self.flatten_if_single_child()
            if error:
                return error
                
            return None
            
        except Exception as e:
            return f"Error during copy operation: {str(e)}"
            
    def flatten_if_single_child(self) -> Optional[str]:
        """
        If destination has only one child folder, move its contents up.
        
        Returns:
            Error message if operation fails, None otherwise
        """
        try:
            # Get immediate children
            children = list(self.destination.iterdir())
            
            # Check if there's exactly one child and it's a directory
            if len(children) == 1 and children[0].is_dir():
                single_child = children[0]
                
                # Create temporary directory for moving
                temp_dir = self.destination.parent / f"{self.destination.name}_temp"
                temp_dir.mkdir()
                
                try:
                    # Move all contents to temp directory
                    for item in single_child.iterdir():
                        shutil.move(str(item), str(temp_dir))
                    
                    # Remove the now-empty child directory
                    single_child.rmdir()
                    
                    # Move everything back from temp to destination
                    for item in temp_dir.iterdir():
                        shutil.move(str(item), str(self.destination))
                        
                    # Remove temp directory
                    temp_dir.rmdir()
                    
                except Exception as e:
                    # Clean up temp directory if it exists
                    if temp_dir.exists():
                        shutil.rmtree(temp_dir)
                    raise e
                    
            return None
            
        except Exception as e:
            return f"Error during structure flattening: {str(e)}"


def copy_paths(source_paths: Union[str, List[str]], destination: str, ignore: List[str] = None) -> Optional[str]:
    """
    Copy multiple paths (files or folders) recursively to a destination while preserving structure.
    
    Args:
        source_paths: Source path or list of source paths (files or folders) to copy
        destination: Destination folder path
        ignore: List of file or folder names or full paths to ignore during copy
        
    Returns:
        Error message if operation fails, None otherwise
    
    Example:
        >>> source_paths = [
        ...     "/path/to/apps/assets",
        ...     "/path/to/apps/configs/config.json",
        ...     "/path/to/apps/context",
        ...     "/path/to/apps/data/org/data.txt"
        ... ]
        >>> ignore_list = [
        ...     "p3.yaml",                           # Ignore by simple file name
        ...     "__pycache__",                       # Ignore by folder name
        ...     "/path/to/apps/toolkit/utils/scripts/export_folders.ipynb"  # Ignore by full path
        ... ]
        >>> error = copy_paths(source_paths, "/path/to/destination", ignore=ignore_list)
        >>> if error:
        >>>     print(f"Copy failed: {error}")
        >>> else:
        >>>     print("Copy completed successfully")
    """
    # Convert single string path to list
    if isinstance(source_paths, str):
        source_paths = [source_paths]
        
    copier = PathCopier(source_paths, destination, ignore)
    return copier.execute()
  
# Example usage
if __name__ == "__main__":
    import packages  # Import your packages module for ROOT_PATH
    
    source_folders = [
        f"{packages.ROOT_PATH}/apps/context",  # Copying entire context directory
        f"{packages.ROOT_PATH}/apps/requirements",
        f"{packages.ROOT_PATH}/apps/services/core.requirements.txt",
        f"{packages.ROOT_PATH}/apps/services/llm/agents/p3",
        f"{packages.ROOT_PATH}/apps/services/bases",
        f"{packages.ROOT_PATH}/apps/toolkit/llm",
        f"{packages.ROOT_PATH}/apps/toolkit/db",
        f"{packages.ROOT_PATH}/apps/toolkit/utils",
        f"{packages.ROOT_PATH}/devops/docker/infra",
        f"{packages.ROOT_PATH}/devops/docker/apps/p3.docker-compose.yaml",
        f"{packages.ROOT_PATH}/devops/docker/services/llm/agents/P3.Dockerfile",
        f"{packages.ROOT_PATH}/front_end/Components",
        f"{packages.ROOT_PATH}/others/docs/projects/P3",
    ]
    destination = f"{packages.ROOT_PATH}/clone"
    
    # Ignore specific files and folders
    ignore_list = [
        "vehicle.yaml", "__pycache__", "node_modules",
        f"{packages.ROOT_PATH}/apps/toolkit/utils/scripts/export_folders.ipynb",
        f"{packages.ROOT_PATH}/devops/docker/apps/vehicle.docker-compose.yaml",
        f"{packages.ROOT_PATH}/front_end/Components/components/vehicle",
        f"{packages.ROOT_PATH}/front_end/Components/components/Face.vue",
        f"{packages.ROOT_PATH}/front_end/Components/components/Map.vue",
        f"{packages.ROOT_PATH}/front_end/Components/components/MultiModalInput.vue",
        f"{packages.ROOT_PATH}/front_end/Components/components/Music.vue",
        f"{packages.ROOT_PATH}/front_end/Components/components/News.vue",
        f"{packages.ROOT_PATH}/front_end/Components/components/Weather.vue",
        f"{packages.ROOT_PATH}/front_end/Components/configs/services_info.json",
        f"{packages.ROOT_PATH}/front_end/Components/pages/AI",
    ]
    
    error = copy_paths(source_folders, destination, ignore=ignore_list)
    if error:
        print(f"Copy failed: {error}")
    else:
        print("Copy completed successfully")

```bash
cd clone
git init
git remote add origin https://p3ds@dev.azure.com/p3ds/P3-Charge.ai/_git/P3-Charge.ai
git remote set-url origin https://ext-hieu.doan:DxBocUBaLa01OZ8Zw2Dby6BS4kXgTUDwCHaW5yTMXRAl60kC9YhnJQQJ99BBACAAAAAs05tvAAASAZDOhPYK@dev.azure.com/p3ds/P3-Charge.ai/_git/P3-Charge.ai

git add .
git commit -m "Initial commit"
git push -u 

git checkout -b fpt/voice-bot/hybrid
git push -u origin fpt/voice-bot/hybrid
```